# 구글 드라이브와 Colab 연동 

In [0]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!ls "/content/gdrive/"

'My Drive'


In [0]:
!ls "/content/gdrive/My Drive/"

In [0]:
!cd "/content/gdrive/My Drive/"

In [0]:
!pwd

/content


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

import os

# 메모리크기 줄이기

[참고커널] [Leonardo Ferreira](https://www.kaggle.com/kabure/extensive-eda-and-modeling-xgb-hyperopt)

In [0]:
def reduce_memory_use(df, verbose=True):
  numerics=['int16','int32', 'int64', 'float16', 'float32', 'float64']
  start_mem = df.memory_usage().sum()/1024**2
  for col in df.columns:
    col_type= df[col].dtypes
    if col_type in numerics:
      c_min = df[col].min() #df[컬럼] 최솟값
      c_max = df[col].max() #df[컬럼] 최댓값
      if str(col_type)[:3]=='int': #컬럼 타입이 int16~ int64 타입일때
        if c_min>np.iinfo(np.int8).min and c_max<np.iinfo(np.int8).max:
          df[col]= df[col].astype(np.int8) #int8로 전환
        elif c_min>np.iinfo(np.int16).min and c_max<np.iinfo(np.int16).max:
          df[col]= df[col].astype(np.int16) #int16으로 전환

        elif c_min>np.iinfo(np.int32).min and c_max<np.iinfo(np.int32).max:
          df[col]= df[col].astype(np.int32) #int32으로 전환

        elif c_min>np.iinfo(np.int64).min and c_max<np.iinfo(np.int64).max:
          df[col]= df[col].astype(np.int64) #int64으로 전환

      else: #str(col_type)[:3]!='int' => float타입
        if c_min>np.finfo(np.float16).min and c_max<np.finfo(np.float16).max:
          df[col]= df[col].astype(np.float16) #float16으로 전환

        elif c_min>np.finfo(np.float32).min and c_max<np.finfo(np.float32).max:
          df[col]= df[col].astype(np.float32) #float32으로 전환

        elif c_min>np.finfo(np.float64).min and c_max<np.finfo(np.float64).max:
          df[col]= df[col].astype(np.float64) #float64으로 전환

  end_mem = df.memory_usage().sum()/ 1024**2
  if verbose:
    print('Memory usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format( end_mem, 100*(start_mem -end_mem)/start_mem))
  return df   

# 데이터셋 읽기

In [0]:
train_transaction=pd.read_csv('/content/gdrive/My Drive/data/train_transaction.csv')
train_identity = pd.read_csv('/content/gdrive/My Drive/data/train_identity.csv')

# test 데이터 읽기
test_transaction = pd.read_csv('/content/gdrive/My Drive/data/test_transaction.csv')
test_identity = pd.read_csv('/content/gdrive/My Drive/data/test_identity.csv')

In [0]:
# reduce_memory_use() 함수를 이용하여 데이터 크기 줄이기
# train 데이터 줄이기
train_transaction = reduce_memory_use(train_transaction)
train_identity= reduce_memory_use(train_identity)

# test 데이터 줄이기
test_transaction = reduce_memory_use(test_transaction)
test_identity= reduce_memory_use(test_identity)

Memory usage decreased to 542.35 Mb (69.4% reduction)
Memory usage decreased to 25.86 Mb (42.7% reduction)
Memory usage decreased to 472.59 Mb (68.9% reduction)
Memory usage decreased to 25.44 Mb (42.7% reduction)


# 데이터 합치기

In [0]:
train_merged = pd.merge(train_transaction, train_identity, on='TransactionID', how='left')
test_merged = pd.merge(test_transaction, test_identity, on='TransactionID', how='left')

# Object / Numerical / Boolean 컬럼 분류

In [0]:
# train_merged 데이터셋에서 
# 데이터 타입별(object, int, float) 컬럼을 분류한다.
object_columns = []
numerical_columns = []
boolean_columns =[]

numerics=['int16','int32', 'int64', 'float16', 'float32', 'float64']


# train_merged는 isFraud가 있는반면 test_merged는 없다.
columns = test_merged.columns

for column in columns:
  column_dtype= train_merged[column].dtypes
  
  # 현재 컬럼의 데이터 타입이 문자열(String)이라면
  if column_dtype=='object':
    object_columns.append(column)
  
  # 현재 컬럼의 데이터 타입이 int/float이라면
  elif column_dtype in numerics:
    numerical_columns.append(column)
    
  # 현재 컬럼의 데이터 타입이 boolean이라면
  elif column_dypte=='boolean':
    boolean_columns.append(column)
    
print('\n---------------------------------------------')
print('numerical_columns(int64, float64) 개수: ', len(numerical_columns))
print('object_columns 개수: ', len(object_columns))
print('boolean_columns 개수: ', len(boolean_columns))


---------------------------------------------
numerical_columns(int64, float64) 개수:  402
object_columns 개수:  31
boolean_columns 개수:  0


In [0]:
object_columns

['ProductCD',
 'card4',
 'card6',
 'P_emaildomain',
 'R_emaildomain',
 'M1',
 'M2',
 'M3',
 'M4',
 'M5',
 'M6',
 'M7',
 'M8',
 'M9',
 'id_12',
 'id_15',
 'id_16',
 'id_23',
 'id_27',
 'id_28',
 'id_29',
 'id_30',
 'id_31',
 'id_33',
 'id_34',
 'id_35',
 'id_36',
 'id_37',
 'id_38',
 'DeviceType',
 'DeviceInfo']

In [0]:
# object_columns 에서 각 컬럼별 클래스 개수를 나타냈다.
for obj_col in object_columns:
  vcnt=train_merged[obj_col].value_counts()
  print('\n[{}]\n{}\n{} class 개수: {}'.format(obj_col, vcnt, obj_col, vcnt.nunique()))
  print('----------------------------------------------------\n')


[ProductCD]
W    439670
C     68519
R     37699
H     33024
S     11628
Name: ProductCD, dtype: int64
ProductCD class 개수: 5
----------------------------------------------------


[card4]
visa                384767
mastercard          189217
american express      8328
discover              6651
Name: card4, dtype: int64
card4 class 개수: 4
----------------------------------------------------


[card6]
debit              439938
credit             148986
debit or credit        30
charge card            15
Name: card6, dtype: int64
card6 class 개수: 4
----------------------------------------------------


[P_emaildomain]
gmail.com           228355
yahoo.com           100934
hotmail.com          45250
anonymous.com        36998
aol.com              28289
comcast.net           7888
icloud.com            6267
outlook.com           5096
msn.com               4092
att.net               4033
live.com              3041
sbcglobal.net         2970
verizon.net           2705
ymail.com             2396


위의 결과는 object 타입의 컬럼들을 대상으로 value_counts() 시킨 결과이다.

위의 결과를 보면 라벨 인코딩하기에 많은 클래스 수를 갖는 컬럼들이 존재한다. 클래스의 개수를 줄일 필요가 있다.

- P_emaildomain(56종류)
- R_emaildomain(55종류)
- id_30 (72종류)
- id_31 (103종류)
- id_33 (74종류)
- DeviceInfo(141종류)


# (1) P_emaildomain

In [0]:
# gmail
train_merged.loc[ train_merged['P_emaildomain'].str.contains('gmail', na=False)]['P_emaildomain'].value_counts()

gmail.com    228355
gmail           496
Name: P_emaildomain, dtype: int64

In [0]:
# yahoo
train_merged.loc[ train_merged['P_emaildomain'].str.contains('yahoo', na=False)]['P_emaildomain'].value_counts()

yahoo.com       100934
yahoo.com.mx      1543
yahoo.fr           143
yahoo.es           134
yahoo.de            74
yahoo.co.uk         49
yahoo.co.jp         32
Name: P_emaildomain, dtype: int64

In [0]:
# outlook
train_merged.loc[train_merged['P_emaildomain'].str.contains('outlook',na=False), 'P_emaildomain'].value_counts()

outlook.com    5096
outlook.es      438
Name: P_emaildomain, dtype: int64

In [0]:
# hotmail
train_merged.loc[train_merged['P_emaildomain'].str.contains('hotmail',na=False), 'P_emaildomain'].value_counts()

hotmail.com      45250
hotmail.es         305
hotmail.fr         295
hotmail.co.uk      112
hotmail.de          43
Name: P_emaildomain, dtype: int64

In [0]:
# live
train_merged.loc[train_merged['P_emaildomain'].str.contains('live',na=False), 'P_emaildomain'].value_counts()


live.com       3041
live.com.mx     749
live.fr          56
Name: P_emaildomain, dtype: int64

In [0]:
# P_emaildomain의 클래스 중 데이터의 개수가 200개
# nunique() 는 컬럼에 속하는 클래스의 개수를 카운트할 때 사용..
train_merged.loc [ train_merged['P_emaildomain'].isin(
    train_merged.P_emaildomain.value_counts()[
        train_merged.P_emaildomain.value_counts()<=200].index), 'P_emaildomain' ]

549                  q.com
641       servicios-ta.com
676                  q.com
879             cfl.rr.com
2273           netzero.net
2555                gmx.de
2614        suddenlink.net
2637        suddenlink.net
2768       frontiernet.net
3043       frontiernet.net
3574        suddenlink.net
8029       frontiernet.net
8056       frontiernet.net
8123       frontiernet.net
8336          cableone.net
8456          cableone.net
8468                gmx.de
8747              yahoo.fr
8832              yahoo.de
9634            cfl.rr.com
9972              yahoo.es
10522       suddenlink.net
10569      frontiernet.net
10740           cfl.rr.com
11124     servicios-ta.com
11139     servicios-ta.com
11144     servicios-ta.com
11153      frontiernet.net
11338      frontiernet.net
11434      frontiernet.net
                ...       
586616               q.com
586942          cfl.rr.com
587182       hotmail.co.uk
587321     frontiernet.net
587354           sc.rr.com
587378     frontiernet.net
5

# (2) R_emaildomain


In [0]:
# yahoo
train_merged.loc[train_merged['R_emaildomain'].str.contains('yahoo', na=False), 'R_emaildomain'].value_counts()

yahoo.com       11842
yahoo.com.mx     1508
yahoo.fr          137
yahoo.de           75
yahoo.es           57
yahoo.co.uk        39
yahoo.co.jp        33
Name: R_emaildomain, dtype: int64

In [0]:
# hotmail
train_merged.loc[train_merged['R_emaildomain'].str.contains('hotmail', na=False), 'R_emaildomain'].value_counts()

hotmail.com      27509
hotmail.fr         293
hotmail.es         292
hotmail.co.uk      105
hotmail.de          42
Name: R_emaildomain, dtype: int64

In [0]:
# live
train_merged.loc[ train_merged['R_emaildomain'].str.contains('live', na=False), 'R_emaildomain'].value_counts()

live.com       762
live.com.mx    754
live.fr         55
Name: R_emaildomain, dtype: int64

In [0]:
# outlook
train_merged.loc[ train_merged['R_emaildomain'].str.contains('outlook', na=False), 'R_emaildomain'].value_counts()

outlook.com    2507
outlook.es      433
Name: R_emaildomain, dtype: int64

# (3) id_30

위의 코드 결과를 보면, 이 컬럼에 존재하는 클래스들은 운영체제(OS) 종류를 나타낸다.

주로 많이 보이는 클래스 iOS, Android, Linux, Mac, Windows 로 보인다.

72개의 클래스 종류가 있으나 일부가 가려져있어서 보기가 어렵다.

(1) 잘보이는 클래스를 대상으로 그룹핑을 한다.

- ex. Mac OS X 10_12_4 , Mac Os X 10.9, ... ==> Mac OS X 
- ex. Windows XP, Windows 7, ...  ==> Windows


일단 아래 먼저 5개의 운영체제를 하나의 그룹으로 다시 묶는다.

- iOS
- Android
- Linux
- Mac OS X
- Windows


In [0]:
# (1) iOS 그룹핑
train_merged.loc[ train_merged['id_30'].str.contains('iOS', na=False), 'id_30']='iOS'
train_merged.id_30.value_counts()

Windows 10          21155
iOS                 19782
Windows 7           13110
Android 7.0          2871
Mac OS X 10_12_6     2559
Mac OS X 10_11_6     2348
Windows 8.1          1914
Mac OS X 10_10_5     1651
Mac OS X 10_13_2     1421
Mac OS X 10_13_1     1211
Linux                1136
Android              1079
Mac OS X 10_13_3      860
Mac OS X 10_13_4      759
Android 7.1.1         630
Mac OS X 10_9_5       570
Android 6.0.1         566
Android 5.1.1         350
Android 8.0.0         343
Mac                   303
Mac OS X 10.13        287
Windows Vista         246
Mac OS X 10.12        218
Mac OS X 10_12_5      217
Mac OS X 10.11        200
Android 8.1.0         192
Windows 8             180
Mac OS X 10.10        135
Windows XP            131
Mac OS X 10_12_4      103
Mac OS X 10_12_3       95
Mac OS X 10_7_5        93
Mac OS X 10_11_5       82
Android 6.0            80
Mac OS X 10_8_5        74
Android 4.4.2          63
Mac OS X 10.9          58
Android 5.0.2          55
Mac OS X 10_

In [0]:
# (2) Mac/ Mac OS X => Mac으로  그룹핑
train_merged.loc[ train_merged['id_30'].str.contains('Mac', na=False), 'id_30']='Mac'
train_merged['id_30'].value_counts()

Windows 10       21155
iOS              19782
Mac              13580
Windows 7        13110
Android 7.0       2871
Windows 8.1       1914
Linux             1136
Android           1079
Android 7.1.1      630
Android 6.0.1      566
Android 5.1.1      350
Android 8.0.0      343
Windows Vista      246
Android 8.1.0      192
Windows 8          180
Windows XP         131
Android 6.0         80
Android 4.4.2       63
Android 5.0.2       55
Android 5.0         41
Android 7.1.2       33
other               15
func                10
Windows              3
Name: id_30, dtype: int64

In [0]:
# (3) Android 그룹핑
train_merged.loc[ train_merged['id_30'].str.contains('Android', na=False), 'id_30']='Android'
train_merged['id_30'].value_counts()

Windows 10       21155
iOS              19782
Mac              13580
Windows 7        13110
Android           6303
Windows 8.1       1914
Linux             1136
Windows Vista      246
Windows 8          180
Windows XP         131
other               15
func                10
Windows              3
Name: id_30, dtype: int64

In [0]:
# (4) Windows 그룹핑
train_merged.loc[ train_merged['id_30'].str.contains('Windows', na=False), 'id_30']='Windows'
train_merged['id_30'].value_counts()

Windows    36739
iOS        19782
Mac        13580
Android     6303
Linux       1136
other         15
func          10
Name: id_30, dtype: int64

# (4) id_31

id_31은 인터넷 브라우저에 대한 정보를 나타낸다.

103개의 클래스가 있다. 

제일 많이 보이는 인터넷 브라우저 이름은 아래의 4개이다.

먼저 아래의 4개를 하나로 통합시켜본다.

- chrome
- safari
- firefox
- Samsung(samsung)

In [0]:
# (1) chrome 그룹핑
train_merged.loc[ train_merged['id_31'].str.contains('chrome', na=False), 'id_31'].value_counts()

chrome 63.0                   22000
chrome 62.0                    7182
chrome 65.0                    6871
chrome 64.0                    6711
chrome 63.0 for android        5806
chrome generic                 4778
chrome 66.0                    4264
chrome 64.0 for android        3473
chrome 65.0 for android        3336
chrome 66.0 for android        2349
chrome 62.0 for android        2097
chrome generic for android     1158
chrome 49.0                     719
chrome 61.0                     642
chrome 61.0 for android         538
chrome                          428
chrome 60.0                     371
chrome 60.0 for android         325
chrome 55.0 for android         302
chrome 58.0 for android         262
chrome 56.0 for android         238
chrome 59.0 for android         227
chrome 63.0 for ios             222
chrome 58.0                     188
chrome 66.0 for ios             159
chrome 52.0 for android         152
chrome 50.0 for android         145
chrome 59.0                 

In [0]:
train_merged.loc[ train_merged['id_31'].str.contains('chrome', na=False), 'id_31']= 'Chrome'
# google, google search application, chromium
train_merged.loc[ train_merged['id_31'].isin(['google','google search application 48.0', 
                                              'google search application 49.0','chromium', 'Google']), 'id_31']='Chrome'
train_merged['id_31'].value_counts()

Chrome                     76206
mobile safari 11.0         13423
mobile safari generic      11474
ie 11.0 for desktop         9030
safari generic              8195
edge 16.0                   4188
firefox 57.0                3315
mobile safari 10.0          2779
edge 15.0                   1600
firefox 59.0                1099
samsung browser 6.2         1061
firefox 58.0                 833
firefox                      673
ie 11.0 for tablet           647
safari 11.0                  550
mobile safari 9.0            541
samsung browser 6.4          509
edge 14.0                    419
firefox 52.0                 394
other                        312
android webview 4.0          285
firefox 60.0                 225
samsung browser generic      210
firefox 48.0                 179
Samsung/SM-G532M             150
opera 49.0                   138
firefox 56.0                 126
safari 10.0                  111
firefox generic              110
samsung browser 7.0          101
          

In [0]:
# (2) safari
train_merged.loc[ train_merged['id_31'].str.contains('safari', na=False), 'id_31']='Safari'
train_merged['id_31'].value_counts()

Chrome                     76206
Safari                     37281
ie 11.0 for desktop         9030
edge 16.0                   4188
firefox 57.0                3315
edge 15.0                   1600
firefox 59.0                1099
samsung browser 6.2         1061
firefox 58.0                 833
firefox                      673
ie 11.0 for tablet           647
samsung browser 6.4          509
edge 14.0                    419
firefox 52.0                 394
other                        312
android webview 4.0          285
firefox 60.0                 225
samsung browser generic      210
firefox 48.0                 179
Samsung/SM-G532M             150
opera 49.0                   138
firefox 56.0                 126
firefox generic              110
samsung browser 7.0          101
android browser 4.0          100
opera                         83
Generic/Android 7.0           81
opera generic                 78
edge 17.0                     74
edge 13.0                     62
          

In [0]:
# (3) opera
train_merged.loc[ train_merged['id_31'].str.contains('opera', na=False), 'id_31']='Opera'
train_merged['id_31'].value_counts()

Chrome                     76206
Safari                     37281
ie 11.0 for desktop         9030
edge 16.0                   4188
firefox 57.0                3315
edge 15.0                   1600
firefox 59.0                1099
samsung browser 6.2         1061
firefox 58.0                 833
firefox                      673
ie 11.0 for tablet           647
samsung browser 6.4          509
Opera                        449
edge 14.0                    419
firefox 52.0                 394
other                        312
android webview 4.0          285
firefox 60.0                 225
samsung browser generic      210
firefox 48.0                 179
Samsung/SM-G532M             150
firefox 56.0                 126
firefox generic              110
samsung browser 7.0          101
android browser 4.0          100
Generic/Android 7.0           81
edge 17.0                     74
edge 13.0                     62
edge                          58
Generic/Android               57
          

In [0]:
# (4) edge
#train_merged.loc[train_merged['id_31'].str.contains('edge', na=False), 'id_31']='MicroSoft'

train_merged.loc[ train_merged['id_31']=='Edge', 'id_31']='MicroSoft'
# Microsoft/Windows 
train_merged.loc[ train_merged['id_31'].str.contains('Microsoft/Windows', na=False), 'id_31']='MicroSoft'
train_merged['id_31'].value_counts()

Chrome                     76206
Safari                     37281
ie 11.0 for desktop         9030
edge 16.0                   4188
firefox 57.0                3315
edge 15.0                   1600
firefox 59.0                1099
samsung browser 6.2         1061
firefox 58.0                 833
firefox                      673
ie 11.0 for tablet           647
samsung browser 6.4          509
Opera                        449
edge 14.0                    419
firefox 52.0                 394
other                        312
android webview 4.0          285
firefox 60.0                 225
samsung browser generic      210
firefox 48.0                 179
Samsung/SM-G532M             150
firefox 56.0                 126
firefox generic              110
samsung browser 7.0          101
android browser 4.0          100
Generic/Android 7.0           81
edge 17.0                     74
edge 13.0                     62
edge                          58
Generic/Android               57
          

In [0]:
# (5) ie
train_merged.loc[ train_merged['id_31'].str.contains('ie', na=False), 'id_31']='IE'
train_merged['id_31'].value_counts()

Chrome                     76206
Safari                     37281
IE                         10018
edge 16.0                   4188
firefox 57.0                3315
edge 15.0                   1600
firefox 59.0                1099
samsung browser 6.2         1061
firefox 58.0                 833
firefox                      673
samsung browser 6.4          509
Opera                        449
edge 14.0                    419
firefox 52.0                 394
other                        312
firefox 60.0                 225
samsung browser generic      210
firefox 48.0                 179
Samsung/SM-G532M             150
firefox 56.0                 126
firefox generic              110
samsung browser 7.0          101
android browser 4.0          100
Generic/Android 7.0           81
edge 17.0                     74
edge 13.0                     62
edge                          58
Generic/Android               57
Samsung/SM-G531H              52
samsung browser 5.4           43
          

In [0]:
# (6) firefox
train_merged.loc[ train_merged['id_31'].str.contains('firefox', na=False), 'id_31' ]='FireFox'
# Mozilla/FireFox 도 같이 합친다.
train_merged.loc[ train_merged['id_31'].isin(['Mozilla/Firefox']), 'id_31']='FireFox'
train_merged['id_31'].value_counts()

Chrome                     76206
Safari                     37281
IE                         10018
FireFox                     7017
edge 16.0                   4188
edge 15.0                   1600
samsung browser 6.2         1061
samsung browser 6.4          509
Opera                        449
edge 14.0                    419
other                        312
samsung browser generic      210
Samsung/SM-G532M             150
samsung browser 7.0          101
android browser 4.0          100
Generic/Android 7.0           81
edge 17.0                     74
edge 13.0                     62
edge                          58
Generic/Android               57
Samsung/SM-G531H              52
samsung browser 5.4           43
samsung browser 5.2           41
samsung browser 4.0           37
MicroSoft                     25
silk                          19
samsung browser 4.2           18
samsung                       16
mobile                        11
ZTE/Blade                      9
samsung br

In [0]:
# (7) samsung
train_merged.loc[ train_merged['id_31'].str.contains('samsung', na=False) , 'id_31'  ]= 'Samsung'
train_merged.loc[ train_merged['id_31'].isin(['Samsung/SM-G532M', 
                                              'Samsung/SM-G531H', 
                                              'Samsung/SCH']), 'id_31']='Samsung'
train_merged['id_31'].value_counts()

Chrome                 76206
Safari                 37281
IE                     10018
FireFox                 7017
edge 16.0               4188
Samsung                 2247
edge 15.0               1600
Opera                    449
edge 14.0                419
other                    312
android browser 4.0      100
Generic/Android 7.0       81
edge 17.0                 74
edge 13.0                 62
edge                      58
Generic/Android           57
MicroSoft                 25
silk                      19
mobile                    11
ZTE/Blade                  9
comodo                     6
maxthon                    6
line                       6
aol                        5
icedragon                  5
Lanix/Ilium                3
palemoon                   2
puffin                     2
waterfox                   2
facebook                   2
M4Tel/M4                   1
iron                       1
Nokia/Lumia                1
Cherry                     1
cyberfox      

In [0]:
# (8) Android
train_merged.loc[ train_merged['id_31'].isin(['android browser 4.0', 'Generic/Android 7.0', 'Generic/Android', 'android']), 'id_31']='Android'
train_merged['id_31'].value_counts()

Chrome         76206
Safari         37281
IE             10018
FireFox         7017
edge 16.0       4188
Samsung         2247
edge 15.0       1600
Opera            449
edge 14.0        419
other            312
Android          239
edge 17.0         74
edge 13.0         62
edge              58
MicroSoft         25
silk              19
mobile            11
ZTE/Blade          9
comodo             6
maxthon            6
line               6
aol                5
icedragon          5
Lanix/Ilium        3
waterfox           2
palemoon           2
facebook           2
puffin             2
LG/K-200           1
Inco/Minion        1
BLU/Dash           1
iron               1
Cherry             1
M4Tel/M4           1
seamonkey          1
cyberfox           1
Nokia/Lumia        1
Name: id_31, dtype: int64

In [0]:
train_merged.loc[ train_merged['id_31'].isin( train_merged.id_31.value_counts().loc[train_merged['id_31'].value_counts().values<=100].index), 'id_31'] ='Others'
train_merged.id_31.value_counts()

Chrome       76206
Safari       37281
IE           10018
FireFox       7017
edge 16.0     4188
Samsung       2247
edge 15.0     1600
Opera          449
edge 14.0      419
other          312
Others         306
Android        239
Name: id_31, dtype: int64

# id_33

화면의 해상도를 나타내는 것 같다.

여기서는 id_30, id_31 처럼 그룹 라벨링을 하지않고 

개수가 500개 미만인 것들을 OtherSize 로 통합시킨다.

In [0]:
train_merged.loc[ train_merged['id_33'].isin(
  train_merged.id_33.value_counts().loc[ train_merged['id_33'].value_counts().values<=500].index), 'id_33'] ='OtherSize'

train_merged.id_33.value_counts()

1920x1080    16874
1366x768      8605
1334x750      6447
OtherSize     5796
2208x1242     4900
1440x900      4384
1600x900      3510
2048x1536     3482
1280x800      2149
2560x1600     2093
2560x1440     1865
2880x1800     1756
1280x1024     1743
1680x1050     1727
1136x640      1712
2436x1125     1484
1280x720      1318
1920x1200     1056
2001x1125     1020
1024x768       824
2220x1080      544
Name: id_33, dtype: int64

# DeviceInfo

object(string) 타입의 데이터를 갖는 컬럼들 중 가장많은 클래스를 갖고있다. (141개)

컴퓨터와같은 기기의 정보를 나타낸다.

- ex) SM, SAMSUNG  → SAMSUNG

- ex) rv:58.0, rv:52.0 →  rv


In [0]:
# (1) Samsung
train_merged['DeviceInfo'].loc[ train_merged['DeviceInfo'].str.contains('SM' , na=False)] = 'Samsung'
train_merged['DeviceInfo'].loc[ train_merged['DeviceInfo'].isin(['SAMSUNG'])]= 'Samsung'
train_merged.DeviceInfo.value_counts()

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


Windows                                47722
iOS Device                             19782
MacOS                                  12573
Samsung                                11933
Trident/7.0                             7440
rv:11.0                                 1901
rv:57.0                                  962
rv:59.0                                  362
ALE-L23 Build/HuaweiALE-L23              312
rv:58.0                                  269
rv:52.0                                  256
Moto G (4) Build/NPJ25.93-14.7           219
hi6210sft Build/MRA58K                   190
rv:48.0                                  178
CAM-L03 Build/HUAWEICAM-L03              172
LG-M700 Build/NMF26X                     169
Moto E (4) Plus Build/NMA26.42-69        160
LG-D693n Build/LRX22G                    150
Moto G (4) Build/NPJS25.93-14-13         147
LG-K410 Build/LRX22G                     136
PRA-LX3 Build/HUAWEIPRA-LX3              135
Moto G (5) Plus Build/NPN25.137-92       130
Moto G Pla

In [0]:
# (2) LG
train_merged['DeviceInfo'].loc[ train_merged['DeviceInfo'].str.contains('LG', na=False)]='LG'
train_merged.DeviceInfo.value_counts()

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


Windows                                47722
iOS Device                             19782
MacOS                                  12573
Samsung                                11933
Trident/7.0                             7440
LG                                      2563
rv:11.0                                 1901
rv:57.0                                  962
rv:59.0                                  362
ALE-L23 Build/HuaweiALE-L23              312
rv:58.0                                  269
rv:52.0                                  256
Moto G (4) Build/NPJ25.93-14.7           219
hi6210sft Build/MRA58K                   190
rv:48.0                                  178
CAM-L03 Build/HUAWEICAM-L03              172
Moto E (4) Plus Build/NMA26.42-69        160
Moto G (4) Build/NPJS25.93-14-13         147
PRA-LX3 Build/HUAWEIPRA-LX3              135
Moto G (5) Plus Build/NPN25.137-92       130
Moto G Play Build/MPIS24.241-15.3-7      129
Moto Z2 Play Build/NPSS26.118-19-14      129
Moto E (4)

In [0]:
# (3) rv
train_merged.loc[ train_merged['DeviceInfo'].str.contains('rv', na=False), 'DeviceInfo']= 'RV'
train_merged.DeviceInfo.value_counts()

Windows                                47722
iOS Device                             19782
MacOS                                  12573
Samsung                                11933
Trident/7.0                             7440
RV                                      4385
LG                                      2563
ALE-L23 Build/HuaweiALE-L23              312
Moto G (4) Build/NPJ25.93-14.7           219
hi6210sft Build/MRA58K                   190
CAM-L03 Build/HUAWEICAM-L03              172
Moto E (4) Plus Build/NMA26.42-69        160
Moto G (4) Build/NPJS25.93-14-13         147
PRA-LX3 Build/HUAWEIPRA-LX3              135
Moto G (5) Plus Build/NPN25.137-92       130
Moto G Play Build/MPIS24.241-15.3-7      129
Moto Z2 Play Build/NPSS26.118-19-14      129
Moto E (4) Build/NMA26.42-69             125
TRT-L53 Build/HUAWEITRT-L53              122
HUAWEI VNS-L53 Build/HUAWEIVNS-L53       113
HUAWEI TAG-L13 Build/HUAWEITAG-L13       113
Moto X Play Build/NPD26.48-24-1          112
HUAWEI VNS

In [0]:
# (4) Moto
train_merged['DeviceInfo'].loc[ train_merged['DeviceInfo'].str.contains('Moto', na=False)]='Moto'
train_merged.DeviceInfo.value_counts()

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


Windows                                 47722
iOS Device                              19782
MacOS                                   12573
Samsung                                 11933
Trident/7.0                              7440
RV                                       4385
Moto                                     2851
LG                                       2563
ALE-L23 Build/HuaweiALE-L23               312
hi6210sft Build/MRA58K                    190
CAM-L03 Build/HUAWEICAM-L03               172
PRA-LX3 Build/HUAWEIPRA-LX3               135
TRT-L53 Build/HUAWEITRT-L53               122
HUAWEI TAG-L13 Build/HUAWEITAG-L13        113
HUAWEI VNS-L53 Build/HUAWEIVNS-L53        113
HUAWEI VNS-L23 Build/HUAWEIVNS-L23        112
WAS-LX3 Build/HUAWEIWAS-LX3               108
Linux x86_64                              102
F3213 Build/36.1.A.1.86                    98
BLADE V8 SE Build/NRD90M                   97
HTC Desire 10 lifestyle Build/MMB29M       94
XT1635-02 Build/NPN26.118-22-2    

In [0]:
# (5) Huawel
train_merged['DeviceInfo'].loc[ train_merged['DeviceInfo'].str.contains('HUAWEI',na=False)]='Huawei'
train_merged.loc[train_merged['DeviceInfo'].isin(['ALE-L23 Build/HuaweiALE-L23']) , 'DeviceInfo']= 'Huawei'
train_merged.DeviceInfo.value_counts()

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


Windows                                 47722
iOS Device                              19782
MacOS                                   12573
Samsung                                 11933
Trident/7.0                              7440
RV                                       4385
Moto                                     2851
LG                                       2563
Huawei                                   2279
hi6210sft Build/MRA58K                    190
Linux x86_64                              102
F3213 Build/36.1.A.1.86                    98
BLADE V8 SE Build/NRD90M                   97
HTC Desire 10 lifestyle Build/MMB29M       94
XT1635-02 Build/NPN26.118-22-2             94
Blade V6 Plus Build/MRA58K                 76
F3113 Build/33.3.A.1.97                    73
CHC-U03 Build/HuaweiCHC-U03                67
HTC Desire 626s Build/LMY47O               67
KFFOWI Build/LVY48F                        65
5080A Build/MRA58K                         63
M4 SS4456 Build/LMY47V            

In [0]:
# Others (200개 적은 클래스들을 통합.)
train_merged.loc[ train_merged['DeviceInfo'].isin(
 train_merged.DeviceInfo.value_counts().loc[ train_merged['DeviceInfo'].value_counts().values<= 200].index 
), 'DeviceInfo']='Others'

train_merged.DeviceInfo.value_counts()

Windows        47722
iOS Device     19782
MacOS          12573
Samsung        11933
Trident/7.0     7440
Others          7138
RV              4385
Moto            2851
LG              2563
Huawei          2279
Name: DeviceInfo, dtype: int64

# Feature Engineering-ObjectColumns

각 object column에 해당하는 클래스의 개수를 줄인다.


In [0]:
def object_feature_engineering(df):
  # P_emaildomain
  df.loc[df['P_emaildomain'].isin(['gmail.com', 'gmail']), 'P_emaildomain']='Gmail'
  
  df.loc[df['P_emaildomain'].isin(['yahoo.com', 'yahoo.com.mx',\
                                                       'yahhoo.es','yahoo.fr',\
                                                       'yahoo.de', 'yahoo.co.uk', \
                                                       'yahoo.co.jp' ]), 'P_emaildomain']='Yahoo'
  
  df.loc[ df['P_emaildomain'].isin(['outlook.com', 'outlook.es']),'P_emaildomain']='Outlook'

  df.loc[ df['P_emaildomain'].isin(['hotmail.com','hotmail.es', 'hotmail.fr', \
                                    'hotmail.co.uk', 'hotmail.de']),'P_emaildomain']='Hotmail'
  
  df.loc[ df['P_emaildomain'].isin(['mac.com', 'icloud.com']), 'P_emaildomain']='Apple'
  
  df.loc[ df['P_emaildomain'].isin(['live.com', 'live.com.mx', 'live.fr']), 'P_emaildomain']='Live'
  
  ## P_emaildomain 중 500개 미만인 것들은 Others로 합친다.
  df.loc[ df['P_emaildomain'].isin( 
      df.P_emaildomain.value_counts()[
          df.P_emaildomain.value_counts()<=500].index), 'P_emaildomain']='Others'
  
  # #################################################################################
  # R_emaildomain
  df.loc[df['R_emaildomain'].isin(['gmail',' gmail.com']), 'R_emaildomain']='Gmail'
  
  df.loc[df['R_emaildomain'].isin(['yahoo.com', 'yahoo.com.mx', 'yahoo.es', 'yahoo.de', \
                                   'yahoo.co.uk', 'yahoo.fr', 'yahoo.co.jp']), 'R_emaildomain']='Yahoo'
  
  df.loc[df['R_emaildomain'].isin(['outlook.com', 'outlook.es']), 'R_emaildomain']='Outlook'
  
  df.loc[df['R_emaildomain'].isin(['hotmail.com', 'hotmail.fr', 'hotmail.es',\
                                   'hotmail.co.uk', 'hotmail.de']), 'R_emaildomain']='Hotmail'
  
  df.loc[df['R_emaildomain'].isin(['icloud.com', 'mac.com']), 'R_emaildomain']='Apple'
  
  df.loc[df['R_emaildomain'].isin(['live.com', 'live.com.mx', 'live.fr']), 'R_emaildomain']='Live'
  
  # R_emaildomain 중 500개 미만인 것들은 Others로 합친다.
  df.loc[ df['R_emaildomain'].isin( 
      df.R_emaildomain.value_counts()[
          df.R_emaildomain.value_counts()<=500].index), 'R_emaildomain']='Others'
  
  # #################################################################################
  # id_30
  ## iOS
  df.loc[ df['id_30'].str.contains('iOS', na=False), 'id_30']='iOS'
  
  ## Mac, Mac OS X
  df.loc[ df['id_30'].str.contains('Mac', na=False), 'id_30']='Mac'
  
  ## Android
  df.loc[ df['id_30'].str.contains('Android', na=False), 'id_30']='Android'
  
  ## Windows
  df.loc[ df['id_30'].str.contains('Windows', na=False), 'id_30']='Windows'
  
  # #################################################################################
  # id_31
  ## chrome, google search application
  df.loc[ df['id_31'].str.contains('chrome', na=False), 'id_31']='Chrome'
  df.loc[ df['id_31'].isin(['google','google search application 48.0', \
                                                'google search application 49.0','chromium']), 'id_31']='Chrome'
  
  ## safari
  df.loc[ df['id_31'].str.contains('safari', na=False), 'id_31']='Safari'
  
  ## firefox, mozila
  df.loc[ df['id_31'].str.contains('firefox', na=False), 'id_31']='Firefox'
  df.loc[ df['id_31'].isin(['Mozilla/Firefox']), 'id_31']='FireFox'
  
  ## samsung
  df.loc[ df['id_31'].str.contains('samsung', na=False) , 'id_31']= 'Samsung'
  df.loc[ df['id_31'].isin(['Samsung/SM-G532M','Samsung/SM-G531H', 'Samsung/SCH']), 'id_31']='Samsung'
  
  ## ie
  df.loc[ df['id_31'].str.contains('ie', na=False), 'id_31']='IE'
  
  ## Edge
  df.loc[ df['id_31'].str.contains('Edge', na=False), 'id_31']='MicroSoft'
  df.loc[ df['id_31'].str.contains('Microsoft/Windows', na=False), 'id_31']='MicroSoft'
  
  ## Opera
  df.loc[ df['id_31'].str.contains('opera', na=False), 'id_31']='Opera'
  
  ## Android
  df.loc[ df['id_31'].isin(['android browser 4.0', 'Generic/Android 7.0', 'Generic/Android', 'android']), 'id_31']='Android'
  
  ## 100개 미만인 브라우저 클래스는 Others로 한다.
  df.loc[ df['id_31'].isin(df.id_31.value_counts().loc[ df['id_31'].value_counts().values<=100].index), 'id_31'] ='Others'
   
  # #################################################################################
  # id_33
  # 500개 미만의 해상도 클래스는 OtherSize로 한다.
  df.loc[ df['id_33'].isin( df.id_33.value_counts().loc[ df['id_33'].value_counts().values<=500].index), 'id_33']='OtherSize'
  
  # #################################################################################
  # DeviceInfo
  ## Samsung
  train_merged['DeviceInfo'].loc[ train_merged['DeviceInfo'].str.contains('SM' , na=False)] = 'Samsung'
  train_merged['DeviceInfo'].loc[ train_merged['DeviceInfo'].isin(['SAMSUNG'])]= 'Samsung'
  train_merged.DeviceInfo.value_counts()
  
  ## LG
  train_merged['DeviceInfo'].loc[ train_merged['DeviceInfo'].str.contains('LG', na=False)]='LG'
  
  ## RV
  train_merged.loc[ train_merged['DeviceInfo'].str.contains('rv', na=False), 'DeviceInfo']= 'RV'
  
  ## Moto
  train_merged['DeviceInfo'].loc[ train_merged['DeviceInfo'].str.contains('Moto', na=False)]='Moto'
  
  ## Huawei
  train_merged['DeviceInfo'].loc[ train_merged['DeviceInfo'].str.contains('HUAWEI',na=False)]='Huawei'
  train_merged.loc[train_merged['DeviceInfo'].isin(['ALE-L23 Build/HuaweiALE-L23']) , 'DeviceInfo']= 'Huawei'
  
  ## Others (100개 미만일 경우의 클래스들)
  train_merged.loc[ train_merged['DeviceInfo'].isin(
  train_merged.DeviceInfo.value_counts().loc[ train_merged['DeviceInfo'].value_counts().values<= 200].index), 'DeviceInfo']='Others'
  
  return df

train_merged=object_feature_engineering(train_merged)
%time


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 6.44 µs


In [0]:
# test_merged 데이터셋 또한 미리 labeling 하기 좋은 형태로 바꾼다.
test_merged= object_feature_engineering(test_merged)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


# Feature Engineering - Null, NaN값 채우기.

In [0]:
# Object 데이터 타입 컬럼에서 NaN값 대신에 다른 string으로 대신하기
def fill_nan(df, obj_columns):
  features= obj_columns
  
  for feature in features:
    df[feature].fillna( 'None', inplace=True)
  
  return df

In [0]:
train_merged=fill_nan(train_merged, object_columns)

In [0]:
train_merged.P_emaildomain.value_counts()

Gmail             228851
Yahoo             102775
None               94456
Hotmail            46005
anonymous.com      36998
aol.com            28289
comcast.net         7888
Apple               6703
Outlook             5534
Others              4611
msn.com             4092
att.net             4033
Live                3846
sbcglobal.net       2970
verizon.net         2705
ymail.com           2396
bellsouth.net       1909
me.com              1522
cox.net             1393
optonline.net       1011
charter.net          816
rocketmail.com       664
mail.com             559
earthlink.net        514
Name: P_emaildomain, dtype: int64

In [0]:
train_merged.R_emaildomain.value_counts()

None             453249
gmail.com         57147
Hotmail           28241
anonymous.com     20529
Yahoo             13691
aol.com            3701
Others             3463
Outlook            2940
comcast.net        1812
Apple              1616
Live               1571
msn.com             852
verizon.net         620
me.com              556
sbcglobal.net       552
Name: R_emaildomain, dtype: int64

In [0]:
train_merged.id_30.value_counts()

None       512975
Windows     36739
iOS         19782
Mac         13580
Android      6303
Linux        1136
other          15
func           10
Name: id_30, dtype: int64

In [0]:
train_merged.id_31.value_counts()

None         450258
Chrome        76206
Safari        37281
IE            10018
FireFox        7017
edge 16.0      4188
Samsung        2247
edge 15.0      1600
Opera           449
edge 14.0       419
other           312
Others          306
Android         239
Name: id_31, dtype: int64

In [0]:
train_merged.DeviceInfo.value_counts()

None           471874
Windows         47722
iOS Device      19782
MacOS           12573
Samsung         11933
Trident/7.0      7440
Others           7138
RV               4385
Moto             2851
LG               2563
Huawei           2279
Name: DeviceInfo, dtype: int64

# LabelEncoding

In [0]:
from sklearn.preprocessing import LabelEncoder

def labelEncoding_features(df, obj_columns):
  # object_columns에 해당하는 컬럼을 레이블인코딩 수행..
  # LabelEncoder를 객체로 생성한 후 fit()과 transform()으로 레이블 인코딩 수행
  features = obj_columns
  
  for feature in features:
    le= LabelEncoder()
    le = le.fit(df[feature])
    df[feature]=le.transform(df[feature])
  
  return df

In [0]:
# label encoder 함수를 이용하여 object를 숫자타입으로 변환시키자.
train_merged= labelEncoding_features(train_merged, object_columns)

# Modeling - LightGBM(LGBM)

In [0]:
# 훈련데이터 셋
y_train = train_merged['isFraud']
X_train = train_merged.drop([ 'TransactionID', 'isFraud'], axis=1)

# 훈련데이터셋에서 학습용(pre_X/y_train), 시험용(pre_X/y_test) 으로 나눈다.
pre_X_train, pre_X_test, pre_y_train, pre_y_test = train_test_split(X_train, 
                                                                    y_train, 
                                                                    test_size=0.2, 
                                                                    random_state=156)


# lgbm 모델 클래스를 부른다.
lgbm_clf = LGBMClassifier(n_estimators=500) # 500번 학습..
evals = [ (pre_X_test, pre_y_test)]
lgbm_clf.fit(pre_X_train, 
             pre_y_train, 
             early_stopping_rounds=100,
             eval_metric='auc',
             eval_set=evals,
             verbose=True
            )

pre_lgbm_roc_score = roc_auc_score(pre_y_test, lgbm_clf.predict_proba(pre_X_test)[:,1], average='macro')
print('pre-training-ROC AUC: {:.4f}'.format(pre_lgbm_roc_score))

[1]	valid_0's auc: 0.803343	valid_0's binary_logloss: 0.129035
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's auc: 0.851446	valid_0's binary_logloss: 0.123012
[3]	valid_0's auc: 0.857778	valid_0's binary_logloss: 0.118833
[4]	valid_0's auc: 0.860174	valid_0's binary_logloss: 0.115555
[5]	valid_0's auc: 0.867529	valid_0's binary_logloss: 0.112351
[6]	valid_0's auc: 0.870808	valid_0's binary_logloss: 0.109955
[7]	valid_0's auc: 0.872056	valid_0's binary_logloss: 0.107817
[8]	valid_0's auc: 0.87528	valid_0's binary_logloss: 0.105806
[9]	valid_0's auc: 0.877114	valid_0's binary_logloss: 0.104226
[10]	valid_0's auc: 0.879373	valid_0's binary_logloss: 0.102584
[11]	valid_0's auc: 0.880525	valid_0's binary_logloss: 0.101211
[12]	valid_0's auc: 0.882033	valid_0's binary_logloss: 0.0999179
[13]	valid_0's auc: 0.883774	valid_0's binary_logloss: 0.0987526
[14]	valid_0's auc: 0.885403	valid_0's binary_logloss: 0.0978287
[15]	valid_0's auc: 0.886502	valid_0's binary_lo

In [0]:
# test_merged 데이터셋의 object컬럼을 라벨인코딩 시키고..
test_merged= labelEncoding_features(test_merged, object_columns)

# test_merged 데이터셋의 object컬럼중 null/nan값을 채우고..
test_merged=fill_nan(test_merged, object_columns)
test_transactionID= test_merged['TransactionID']

X_test = test_merged.drop('TransactionID', axis=1)

# 테스트 데이터셋 예측결과 - submission파일의 isFraud 에 해당.
lgbm_pred = lgbm_clf.predict_proba(X_test)[:,1]

#결과 데이터 프레임을 생성한다.
data= { 'TransactionID': test_transactionID, 
       'isFraud': lgbm_pred
      }
lgbm_result = pd.DataFrame(data)

# 데이터의 크기를 줄인다.
lgbm_result=reduce_memory_use(lgbm_result)

# lgbm_result={ 'TransactionID': test_transactionID,
#               'isFraud': lgbm_pred
#             }
# lgbm_result= pd.DataFrame(lgbm_result)



# csv파일을 생성한다.
lgbm_result.to_csv('lgbm_submission.csv', index=False)

Memory usage decreased to  6.77 Mb (30.0% reduction)


In [0]:
!ls

adc.json  gdrive  lgbm_submission.csv  sample_data


In [0]:
!cp lgbm_submission.csv "/content/gdrive/My Drive/."

In [0]:
!ls "/content/gdrive/My Drive/"

'음성 녹음_001.m4a'
'00학과_학번이름_결과보고서 및 만족도 조사(2018)-1.hwp'
'음성 녹음_002.m4a'
'음성 녹음_003.m4a'
 과제11_이진트리프로그래밍.hwp
'19.03.29. 최은강_마일리지노트.pdf'
'마일리지 장학금 인정 행사 2018.2학기_게시용.hwp'
 20190525_203151.jpg
 20190525_204330.jpg
 alakazam.png
 ArduinoHardware.h
'Colab Notebooks'
 data
 garden0.ipynb
 homeworkend.png
 자기소개서_이력서_최은강_수정.hwp
 ieee_fraud_detection_inColab_myself.ipynb
 참가동의서.jpg
 lgbm_submission.csv
 최은강.pdf
 review0604.ipynb
 robot_msgs.zip
 Screenshot_2019-04-20-14-37-08.jpg
 sketch1528037051402.png
 sketch1528037382445.png
 sketch1528537207985.png
 sketch1530804079146.png
 sketch1536518259484.png
 sketch1536518300205.png
 sketch1547399229927.png
 sketch1548565861693.png
 sketch1555481816221.png
 Sketch829431.jpg
 SSD512.zip
 Untitled0.ipynb
 Untitled1.ipynb
 Untitled2.ipynb
 Untitled3.ipynb
 VL53L0X.zip
